In [62]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

In [63]:
import os

In [64]:
def new_titanic_data():
    sql_query = 'SELECT * FROM passengers'
    df = pd.read_sql(sql_query, get_connection('titanic_db'))
    df.to_csv('titanic_df.csv')
    return df

In [65]:
def get_titanic_data(cached=False):
    if cached or os.path.isfile('titanic_df.csv') == False:
        df = new_titanic_data()
    else:
        df = pd.read_csv('titanic_df.csv', index_col=0)
    return df

In [66]:
titanic = get_titanic_data()

In [6]:
titanic.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [67]:
def titanic_split(df):

    train_validate, test = train_test_split(df, test_size=.2,
                                        random_state=123,
                                        stratify=df.survived)
    train, validate = train_test_split(train_validate, test_size=.3,
                                        random_state=123,
                                        stratify=train_validate.survived)
    return train, validate, test

def impute_mean_age(train, validate, test):

    imputer = SimpleImputer(strategy = 'mean')

    train['age'] = imputer.fit_transform(train[['age']])

    validate['age'] = imputer.transform(validate[['age']])

    test['age'] = imputer.transform(test[['age']])

    return train, validate, test

def prep_titanic_data(cached=True):
#     df = get_titanic_data(cached)
    df = df[~df.embarked.isnull()]
    titanic_dummies = pd.get_dummies(df.embarked, drop_first=True)
    df = pd.concat([df, titanic_dummies], axis=1)
    df = df.drop(columns='deck')

    train, validate, test = titanic_split(df)

    train, validate, test = impute_mean_age(train, validate, test)

    return train, validate, test

In [68]:
 titanic = titanic[~titanic.embarked.isnull()]

In [69]:
titanic.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [70]:
titanic_dummies = pd.get_dummies(titanic[['sex', 'embark_town', 'class']], drop_first=True)


In [71]:
titanic = pd.concat([titanic, titanic_dummies], axis=1)


In [72]:
titanic = titanic.drop(columns=['embark_town','class', 'sex', 'pclass'])


In [73]:
titanic.head()

,passenger_id,survived,age,sibsp,parch,fare,embarked,deck,alone,sex_male,embark_town_Queenstown,embark_town_Southampton,class_Second,class_Third
0,0,0,22.0,1,0,7.2500,S,NaN,0,1,0,1,0,1
1,1,1,38.0,1,0,71.2833,C,C,0,0,0,0,0,0
2,2,1,26.0,0,0,7.9250,S,NaN,1,0,0,1,0,1
3,3,1,35.0,1,0,53.1000,S,C,0,0,0,1,0,0
4,4,0,35.0,0,0,8.0500,S,NaN,1,1,0,1,0,1


In [74]:
titanic = titanic.drop(columns='deck')


In [75]:
train, validate, test = titanic_split(titanic)


In [76]:
train.age.isnull().values.any()

True

In [77]:
train, validate, test = impute_mean_age(train, validate, test)


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [78]:
train.shape

(497, 13)

In [79]:
validate.shape

(214, 13)

In [80]:
test.shape

(178, 13)

In [81]:
test.head()

,passenger_id,survived,age,sibsp,parch,fare,embarked,alone,sex_male,embark_town_Queenstown,embark_town_Southampton,class_Second,class_Third
561,561,0,40.000000,0,0,7.8958,S,1,1,0,1,0,1
328,328,1,31.000000,1,1,20.5250,S,0,0,0,1,0,1
643,643,1,29.916875,0,0,56.4958,S,1,1,0,1,0,1
498,498,0,25.000000,1,2,151.5500,S,0,0,0,1,0,0
875,875,1,15.000000,0,0,7.2250,C,1,0,0,0,0,1


In [82]:
train.survived.value_counts(normalize=True)


0    0.617706
1    0.382294
Name: survived, dtype: float64

In [83]:
logit = LogisticRegression()


In [84]:
X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [85]:
logit = logit.fit(X_train, y_train)


ValueError: could not convert string to float: 'C'

In [27]:
X_train.columns


Index(['passenger_id',          'age',        'sibsp',        'parch',
               'fare',     'embarked',         'deck',        'alone',
                    2,              3],
      dtype='object')

In [29]:
logit.coef_


AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [ ]:
y_pred = logit.predict(X_train)
y_pred_proba = logit.predict_proba(X_train)

In [ ]:
y_pred_proba

In [ ]:
logit.score(X_train, y_train)


In [ ]:
print(confusion_matrix(y_train, y_pred))


In [ ]:
print(classification_report(y_train, y_pred))


In [34]:
titanic2 = get_titanic_data()

In [35]:
titanic2.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [36]:
titanic2.age.isnull().values.any()

True

Create another model that includes age in addition to fare and pclass. Does this model perform better than your previous one?

In [37]:
titanic2 = titanic2.drop(columns=['sex', 'sibsp', 'parch', 'embarked','class', 'deck','embark_town', 'alone'])


In [38]:
titanic2.head()

,passenger_id,survived,pclass,age,fare
0,0,0,3,22.0,7.2500
1,1,1,1,38.0,71.2833
2,2,1,3,26.0,7.9250
3,3,1,1,35.0,53.1000
4,4,0,3,35.0,8.0500


In [39]:
titanic2 = titanic2.drop(columns='passenger_id')

In [40]:
titanic2.head()

,survived,pclass,age,fare
0,0,3,22.0,7.2500
1,1,1,38.0,71.2833
2,1,3,26.0,7.9250
3,1,1,35.0,53.1000
4,0,3,35.0,8.0500


In [41]:
titanic2_dummies = pd.get_dummies(titanic2.pclass, drop_first=True)


In [42]:
titanic2 = pd.concat([titanic2, titanic2_dummies], axis=1)


In [43]:
titanic2.head()

,survived,pclass,age,fare,2,3
0,0,3,22.0,7.2500,0,1
1,1,1,38.0,71.2833,0,0
2,1,3,26.0,7.9250,0,1
3,1,1,35.0,53.1000,0,0
4,0,3,35.0,8.0500,0,1


In [44]:
train, validate, test = titanic_split(titanic2)


In [45]:
train.head()

,survived,pclass,age,fare,2,3
583,0,1,36.0,40.1250,0,0
165,1,3,9.0,20.5250,0,1
50,0,3,7.0,39.6875,0,1
259,1,2,50.0,26.0000,1,0
306,1,1,NaN,110.8833,0,0


In [46]:
train.isnull().values.any()

True

In [47]:
impute_mean_age(train, validate, test)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(     survived  pclass        age      fare  2  3
 583         0       1  36.000000   40.1250  0  0
 165         1       3   9.000000   20.5250  0  1
 50          0       3   7.000000   39.6875  0  1
 259         1       2  50.000000   26.0000  1  0
 306         1       1  29.678105  110.8833  0  0
 ..        ...     ...        ...       ... .. ..
 313         0       3  28.000000    7.8958  0  1
 636         0       3  32.000000    7.9250  0  1
 222         0       3  51.000000    8.0500  0  1
 485         0       3  29.678105   25.4667  0  1
 744         1       3  31.000000    7.9250  0  1
 
 [498 rows x 6 columns],
      survived  pclass        age      fare  2  3
 610         0       3  39.000000   31.2750  0  1
 424         0       3  18.000000   20.2125  0  1
 568         0       3  29.678105    7.2292  0  1
 334         1       1  29.678105  133.6500  0  0
 101         0       3  29.678105    7.8958  0  1
 ..        ...     ...        ...       ... .. ..
 176         0       3 

In [48]:
train.shape

(498, 6)

In [50]:
train.age.isnull().values.any()

False

In [51]:
train.survived.value_counts(normalize=True)


0    0.616466
1    0.383534
Name: survived, dtype: float64

In [52]:
logit = LogisticRegression()


In [53]:
X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [54]:
logit = logit.fit(X_train, y_train)


In [55]:
X_train.columns


Index(['pclass', 'age', 'fare', 2, 3], dtype='object')

In [56]:
print(logit.coef_)

[[-0.67260505 -0.02902885  0.00259074  0.14850399 -0.41036784]]


In [57]:
y_pred = logit.predict(X_train)
y_pred_proba = logit.predict_proba(X_train)

In [58]:
y_pred_proba


array([[0.40808026, 0.59191974],
       [0.65716147, 0.34283853],
       [0.63250245, 0.36749755],
       [0.64455554, 0.35544446],
       [0.32327957, 0.67672043],
       [0.50494894, 0.49505106],
       [0.59660579, 0.40339421],
       [0.49917841, 0.50082159],
       [0.78313474, 0.21686526],
       [0.75471528, 0.24528472],
       [0.8292616 , 0.1707384 ],
       [0.58902943, 0.41097057],
       [0.5868048 , 0.4131952 ],
       [0.48350205, 0.51649795],
       [0.5891301 , 0.4108699 ],
       [0.84916005, 0.15083995],
       [0.78300271, 0.21699729],
       [0.78307788, 0.21692212],
       [0.54101014, 0.45898986],
       [0.7591043 , 0.2408957 ],
       [0.76808348, 0.23191652],
       [0.78312374, 0.21687626],
       [0.2549033 , 0.7450967 ],
       [0.30008336, 0.69991664],
       [0.48113939, 0.51886061],
       [0.74834275, 0.25165725],
       [0.38689257, 0.61310743],
       [0.51601041, 0.48398959],
       [0.37317122, 0.62682878],
       [0.4903032 , 0.5096968 ],
       [0.

In [59]:
logit.score(X_train, y_train)


0.6847389558232931

In [60]:
print(confusion_matrix(y_train, y_pred))


[[247  60]
 [ 97  94]]


In [61]:
print(classification_report(y_train, y_pred))


              precision    recall  f1-score   support

           0       0.72      0.80      0.76       307
           1       0.61      0.49      0.54       191

    accuracy                           0.68       498
   macro avg       0.66      0.65      0.65       498
weighted avg       0.68      0.68      0.68       498



68% Accuracy